# 2. Single Node with multiple inputs

In [41]:
from langgraph.graph import StateGraph, START, END

In [42]:
from typing import Dict, TypedDict, List, Literal

In [43]:
class AgentState(TypedDict):
    number1: int
    number2: int
    number3: int
    number4: int
    operation1: str
    operation2: str
    finalnumber1: int
    finalnumber2: int

# Node

In [44]:
def add_node1(state: AgentState) -> AgentState:
    """ This node computes the sum of first 2 numbers"""
    state['finalnumber1'] = state['number1'] + state['number2']
    return state

def subtract_node1(state: AgentState) -> AgentState:
    """ This node computes the difference of first 2 numbers"""
    state['finalnumber1'] = state['number1'] - state['number2']  # Fixed: use finalnumber1
    return state

def add_node2(state: AgentState) -> AgentState:
    """ This node computes the sum of second 2 numbers"""
    state['finalnumber2'] = state['number3'] + state['number4']  # Fixed: use finalnumber2
    return state

def subtract_node2(state: AgentState) -> AgentState:
    """ This node computes the difference of second 2 numbers"""
    state['finalnumber2'] = state['number3'] - state['number4']
    return state

def decide_next_node1(state: AgentState) -> str:  # Fixed: return str, not AgentState
    """This node will select the next phase for operation1"""
    if state["operation1"] == "+":
        return "addition_operation1"
    elif state["operation1"] == "-":
        return "subtraction_operation1"
    else:
        return "addition_operation1"  # Default case

def decide_next_node2(state: AgentState) -> str:  # Fixed: return str, not AgentState
    """This node will select the next phase for operation2"""
    if state["operation2"] == "+":
        return "addition_operation2"
    elif state["operation2"] == "-":
        return "subtraction_operation2"
    else:
        return "addition_operation2"  # Default case

# Graph

In [45]:
graph = StateGraph(AgentState)

graph.add_node("add_node1", add_node1)
graph.add_node("subtract_node1", subtract_node1)
graph.add_node("router1", lambda state:state)

graph.add_edge(START, "router1")
graph.add_conditional_edges(
    "router1", 
    decide_next_node1,
    {
        # Edge: Node format
        "addition_operation1": "add_node1",
        "subtraction_operation1": "subtract_node1"
    }
)

graph.add_edge("add_node1", "router2")
graph.add_edge("subtract_node1", "router2")

graph.add_node("add_node2", add_node2)
graph.add_node("subtract_node2", subtract_node2)
graph.add_node("router2", lambda state:state)

graph.add_conditional_edges(
    "router2",
    decide_next_node2,
    {
        # Edge: Node format
        "addition_operation2": "add_node2",
        "subtraction_operation2": "subtract_node2"
    }
)

graph.add_edge("add_node2", END)
graph.add_edge("subtract_node2", END)

app=graph.compile()

In [ ]:
result = app.invoke({"number1":10,"number2":10, "number3":20,"number4":5,"operation1":"+","operation2":"-"},"final")

In [49]:
print(result["finalnumber1"], result["finalnumber2"])

20 15
